# web of science 学科共现

In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
#import matplotlib.pyplot as plt
os.chdir('D:/software/0install package/VOSviewer')

In [ ]:
#data = pd.read_excel("D:/software/0install package/VOSviewer/web tab.xlsx")
data = pd.read_table("WOS tab0304.txt",sep="	",index_col=False)
df_wc = data[data.WC.str.contains('; ')] #筛选包含分隔符的数据，web of science是分号+空格,只有有分割符才代表是出现了两个及以上学科
print(len(data))
print(len(df_wc))#对比两个数字查看比例，大部分论文(接近50%)只有一个学科，所以学科共现不一定有意义

In [ ]:
data.head()#数据示例

In [ ]:
xueke_list = []
for wc in df_wc.WC:
    wc_list = wc.split("; ")
    xueke_list.extend(wc_list)
xueke_list_unique = list(set(xueke_list))
print(len(xueke_list_unique))

In [ ]:
xueke_list_unique#web of science上边导出的学科包含二级学科，如果想用一级学科的话需要替换学科名，把逗号后边的二级删掉

In [ ]:
xueke_frame = pd.DataFrame(np.zeros([len(xueke_list_unique),len(xueke_list_unique)]),columns=xueke_list_unique,index=xueke_list_unique)
for wc in df_wc.WC:
    wc_list = wc.split('; ')
    for i in range(len(wc_list)-1):
        xueke_frame.loc[wc_list[i]][wc_list[i+1]]=xueke_frame.loc[wc_list[i]][wc_list[i+1]]+1

for j in range(len(xueke_frame)):#对角线都为0
    xueke_frame.iloc[i,i]=0

In [ ]:
g = nx.Graph()

In [ ]:
g = nx.Graph()
for i in range(len(xueke_list_unique)-1):
    xueke = xueke_list_unique[i]
    for j in range(i+1,len(xueke_list_unique)):
        w=0
        xueke2 = xueke_list_unique[j]
        w = xueke_frame.loc[xueke][xueke2]+xueke_frame.loc[xueke2][xueke]
        g.add_edge(xueke,xueke2,weight=w)
nx.write_pajek(g,"D:/software/0install package/VOSviewer/web tab.net")

In [ ]:
# import matplotlib.pyplot as plt
# nx.draw(g)
# plt.show()
nx.write_pajek(g,"D:/software/0install package/VOSviewer/web tab.net")
#将net文件导入vosviewer

# 中国知网-共现

In [111]:
import os
import pandas as pd
import numpy as np
import networkx as nx
#import matplotlib.pyplot as plt
os.chdir('D:/software/0install package/VOSviewer/todo_CNKI')

In [112]:
data = pd.read_excel("CNKI.xlsx")
data.columns

Index(['SrcDatabase-来源库', 'Title-题名', 'Author-作者', 'Organ-单位', 'Source-文献来源',
       'Keyword-关键词'],
      dtype='object')

In [113]:
data.columns = (['SrcDatabase', 'Title', 'Author', 'Organ', 'Source','Keyword'])

In [117]:
organ_list = []
for organ in data.Author:
    organlist = organ.strip(';').split(";")#作者和机构
    #organlist = organ.split(";;")#关键词
    organ_list.extend(organlist)
organ_list_unique = list(set(organ_list))
print(len(organ_list_unique))

789


In [118]:
organ_frame = pd.DataFrame(np.zeros([len(organ_list_unique),len(organ_list_unique)]),columns=organ_list_unique,index=organ_list_unique)
for organ in data.Author:
    organlist = organ.strip(';').split(";")#作者和机构
    #organlist = organ.split(";;")#关键词
    if len(organlist)>1:
        for i in range(len(organlist)-1):
            organ_frame.loc[organlist[i]][organlist[i+1]]=organ_frame.loc[organlist[i]][organlist[i+1]]+1

for j in range(len(organ_frame)):#对角线都为0
    organ_frame.iloc[j,j]=0

In [119]:
g = nx.Graph()
for i in range(len(organ_list_unique)-1):
    organ = organ_list_unique[i]
    for j in range(i+1,len(organ_list_unique)):
        w=0
        organ2 = organ_list_unique[j]
        w = organ_frame.loc[organ][organ2]+organ_frame.loc[organ2][organ]
        if w>0:
            g.add_edge(organ,organ2,weight=w)

KeyboardInterrupt: 

In [ ]:
nx.write_pajek(g,"D:/software/0install package/VOSviewer/todo_CNKI/cnki_author.net")
#将net文件导入vosviewer

In [ ]:
import matplotlib.pyplot as plt
nx.draw(g)
plt.show()